In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from menu_functions import menu_to_pdf

import time

def rag_gpt():
    
    t1 = time.time()
    path = "C:/Users/busch/OneDrive/Documents/Fac/M2/UE1 - Advanced programming and data visualization/Advanced Programming/projet/environment/"
    
    load_dotenv(f"{path}.env")
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    
    context = False
    question = ""
    
    print("TIME : Load llm and environment : ", time.time() - t1)
    
    while question != "Exit":
        question = input() # Save the question of the user
        if context == False:
            
            t2 = time.time()
            # Scraping du menu du restaurant
            menu_to_pdf(question)
            print("TIME : Scrap the menu and implement context : ", time.time() - t2)
            
            # Connection au contexte
            t3 = time.time()
            file_path = f"{path}menu.pdf"
            loader = PyPDFLoader(file_path)

            docs = loader.load()

            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
            splits = text_splitter.split_documents(docs)
            vectorstore = InMemoryVectorStore.from_documents(
                documents=splits, embedding=OpenAIEmbeddings()
            )

            retriever = vectorstore.as_retriever()

            # Prompt
            system_prompt = (
                "You are an assistant for question-answering tasks about the menu of a specific restaurant. "
                "Translate the menu in the language of the question. "
                "A give you a menu, just answer my question, and don't take attention to the name that I specified. "
                "Use the following pieces of retrieved context to answer "
                "the question. If you don't know the answer, say that you "
                "don't know. Use three sentences maximum and keep the "
                "answer concise."
                "\n\n"
                "{context}"
            )

            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", system_prompt),
                    ("human", "{input}"),
                ]
            )

            question_answer_chain = create_stuff_documents_chain(llm, prompt)
            rag_chain = create_retrieval_chain(retriever, question_answer_chain)
            
            context = True

            print("TIME : Connect to the context : ", time.time() - t3)
            
        if question != "Exit":
            # Write an answer from the LLM
            t4 = time.time()
            results = rag_chain.invoke({"input": question})

            # Print the answer
            print(results["answer"])
            print("TIME : Answer the question : ", time.time() - t4)
            

C:\Users\busch\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
rag_gpt()

TIME : Load llm and environment :  0.06978321075439453


 What is the menu at Esplanade ?


TIME : HF_model :  4.052068471908569
TIME : resto_link :  3.899329423904419
TIME : get_link() :  7.951397895812988
TIME : ScrapMenuToday() :  79.98355531692505
TIME : clean_menu() :  0.0
TIME : get_menu(question) :  87.9359622001648
TIME : Scrap the menu and implement context :  88.3443775177002
TIME : Connect to the context :  2.137829065322876
The menu at Esplanade includes student menus on the 1st and 2nd floors. On the 2nd floor, you can find dishes like chicken fillet with homemade forest sauce, green beans, and fries. On the 1st floor, they offer homemade fish curry with twists and vegetarian shepherd's pie. The dessert options include vanilla mille-feuille, coconut tart, and Black Forest cake.
TIME : Answer the question :  2.3824915885925293


 Exit


In [3]:
rag_gpt()

TIME : Load llm and environment :  0.07453393936157227


 What is the menu at Esplanade ?


TIME : get_link() :  10.289774894714355
TIME : ScrapMenuToday() :  25.57329821586609
TIME : clean_menu() :  423.0919601917267
TIME : get_menu(question) :  458.9560377597809
TIME : Scrap the menu and implement context :  458.9985580444336
TIME : Connect to the context :  3.0064380168914795
The menu at Esplanade includes dishes like aiguillettes of wheat and emmental, chou romanesco with herbs of Provence, homemade tartiflette, vegetable wok, Chinese noodles, paëlla with homemade green vegetables, and vegetable soup of the day with carrots and celery. Additionally, there are desserts like Norman tart, thousand sheets, and cake nature with chocolate nuggets available on the 1st and 2nd floors. The Crystal Shop offers dishes like wheat chips, chou romanesco, fries, vegetable wok, and Chinese noodles for on-site consumption or take-out.
TIME : Answer the question :  2.9641318321228027


 Exit
